In [1]:
import pandas as pd

In [46]:
file = 'raw/city-county-prop-tax-analysis.xlsx'
dtype = {
    'County #': str
}
raw_tax = pd.read_excel(file, dtype=dtype, sheet_name='TaxesByCity')
raw_res_tax = pd.read_excel(file, dtype=dtype, sheet_name='Class4ResTaxesByCity')
raw_total_per_cap = pd.read_excel(file, dtype=dtype, sheet_name='City Total Per Capita')
raw_res_per_cap = pd.read_excel(file, dtype=dtype, sheet_name='City Res Per Capita')

In [47]:
raw_tax = raw_tax[:128]
raw_res_tax = raw_res_tax[:128]
raw_total_per_cap = raw_total_per_cap[:128]
raw_res_per_cap = raw_res_per_cap[:128].copy()
raw_res_per_cap.drop(
    labels=['$ Change', '% Change'], 
    axis=1,
    inplace=True,
)

In [48]:
df_total_tax = raw_tax.melt(
    id_vars=['County #', 'County','City Code','City'],
    var_name='year',
    value_name='total_taxes'
)
df_res_tax = raw_res_tax.melt(
    id_vars=['County #', 'County','City Code','City'],
    var_name='year',
    value_name='residential_taxes'
)
df_total_per_cap = raw_total_per_cap.melt(
    id_vars=['County #', 'County','City Code','City'],
    var_name='year',
    value_name='total_taxes_per_capita'
)
df_res_per_cap = raw_res_per_cap.melt(
    id_vars=['County #', 'County','City Code','City'],
    var_name='year',
    value_name='residential_taxes_per_capita'
)

In [49]:
combined = df_total_tax.merge(df_res_tax[['City','year','residential_taxes']], on=['City','year'])
combined = combined.merge(df_total_per_cap[['City','year','total_taxes_per_capita']], on=['City', 'year'])
combined = combined.merge(df_res_per_cap[['City','year','residential_taxes_per_capita']], on=['City','year'])

# calculating new data fields
combined['res_percent'] = combined['residential_taxes'] / combined['total_taxes']
combined['non_res_taxes'] = combined['total_taxes'] - combined['residential_taxes']
combined['non_res_taxes_per_capita'] = combined['total_taxes_per_capita'] - combined['residential_taxes_per_capita']

print(len(combined), 'rows')
combined.head()

2176 rows


,County #,County,City Code,City,year,total_taxes,residential_taxes,total_taxes_per_capita,residential_taxes_per_capita,res_percent,non_res_taxes,non_res_taxes_per_capita
0,54,Mineral,ALB-54,Alberton,2002,1.696918e+05,1.028304e+05,457.390318,277.170970,0.605983,66861.378080,180.219348
1,30,Deer Lodge,ANA-30,Anaconda,2002,2.391263e+06,1.638065e+06,258.850721,177.318092,0.685021,753198.427280,81.532629
2,17,Roosevelt,BAN-17,Bainville,2002,4.877227e+04,1.993519e+04,299.216377,122.301769,0.408740,28837.081130,176.914608
3,39,Fallon,BAK-39,Baker,2002,4.726485e+05,2.348727e+05,288.200275,143.215066,0.496929,237775.741756,144.985208
4,10,Carbon,BER-10,Bearcreek,2002,3.647994e+04,2.910489e+04,434.284982,346.486804,0.797833,7375.046949,87.798178


In [50]:
combined.to_csv('data/taxes-by-city.csv', index=False)